In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import docx
import time

# 获取URL

In [2]:
url = 'https://www.xbiquge.cc/book/49549/35899003.html' #测试用 章节
main_url =  'https://www.xbiquge.cc/book/49549/'       #豪婿
cache = '36254785.html'
#main_url = 'https://www.xbiquge.cc/book/420/'           #龙王传说
# main_url = 'https://www.xbiquge.cc/book/7177/'          #阴阳冕
#main_url ='https://www.xbiquge.cc/book/46844/'          #终极斗罗
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3314.0 Safari/537.36 SE 2.X MetaSr 1.0'
}

In [3]:
t = time.time()
while True:
    try:
        #加入timeout防止单次访问用时过长，timeout = 5防止对服务器造成过大负担
        main_response = requests.get(main_url, headers = header, timeout = 5)
        break
    except:
        print('---URL {} Failed...Try to connect again'.format(main_url))
print('---------Done!: used {:.2f}s--------'.format(time.time() - t))

---------Done!: used 1.58s--------


# 解码 乱码->中文

In [4]:
#www.xbiquge.cc 使用gbk进行编码，所以需要gbk解码
main_response.encoding = "gbk"
main_text = main_response.content.decode('gbk')

# 读入beautifulsoup进行解析

In [5]:
#html.parser是解析器，也可是lxml
main_soup = BeautifulSoup(main_text,'lxml')
#把整个网页转换成解析过的String，不过感觉不需要这一步
#直接使用main_text可以达到同样效果
strMain = main_soup.prettify()
#获得主标题
find_title = re.compile(r'<meta content="(.*?)" property="og:title"/>')
#获得作者
find_author = re.compile(r'<meta content="(.*?)" property="og:novel:author"/>')
mainTitle = find_title.findall(strMain)[0]
author = find_author.findall(strMain)[0]

# 获取小说list

In [6]:
#解析主网页的小说列表，笔趣阁都是以同一个形式加载的
pattern = re.compile(r'<a href="(\d+.html)">')
text_list = pattern.findall(main_soup.prettify())

In [7]:
#test block
text_list

['31073090.html',
 '31073091.html',
 '31073092.html',
 '31073093.html',
 '31073094.html',
 '31073095.html',
 '31073096.html',
 '31073097.html',
 '31073098.html',
 '31073099.html',
 '31073100.html',
 '31073101.html',
 '31073102.html',
 '31073103.html',
 '31073104.html',
 '31073105.html',
 '31073106.html',
 '31073107.html',
 '31073108.html',
 '31073109.html',
 '31073110.html',
 '31073111.html',
 '31073112.html',
 '31073113.html',
 '31073114.html',
 '31073115.html',
 '31073116.html',
 '31073117.html',
 '31073118.html',
 '31073119.html',
 '31073120.html',
 '31073121.html',
 '31073122.html',
 '31073123.html',
 '31073124.html',
 '31073125.html',
 '31073126.html',
 '31073127.html',
 '31073128.html',
 '31073129.html',
 '31073130.html',
 '31073131.html',
 '31073132.html',
 '31073133.html',
 '31073134.html',
 '31073136.html',
 '31073137.html',
 '31073138.html',
 '31073139.html',
 '31073140.html',
 '31073141.html',
 '31073142.html',
 '31073143.html',
 '31073144.html',
 '31073145.html',
 '31073146

# 每一个章节分别进行提取输出操作

   ## 1. 读取每一个章节的内容和标题

In [8]:
def write_to_doc(book):
    begin_time = time.time()
    chapter_url = main_url+book
    print('---Getting URL...---')
    while True:
        try:
            #加timeout防单词用时过长
            chapter_response = requests.get(chapter_url,headers = header,timeout = 5)
            break
        except:
            print('---URL {} Failed...Try to connect again'.format(book))
            continue
    print('-----URL {} used {:.2f}s-----'.format(book, time.time() - begin_time))
    #print(chapter_response.content)
    #小说主体部分需要重新解码
#     try:
    chapter_response.encoding = "gbk"
    chapter_text = chapter_response.content.decode('gbk')
#     except:
#         print("Missing URL:{}".format(book))
#         return "Missing URL:{}".format(book)", "Missing URL:{}".format(book)"
    #以beautiful的形式读入，方便后续解析
    chapter_soup = BeautifulSoup(chapter_text,'lxml')

    #通过分析页面，笔趣阁小说主体部分在 id为content的div里
    div = chapter_soup.find_all('div',id='content')#,string='更多'
    #将获得的tag们全部变成string
    div_text =div[0].prettify()
    #网页中每个段落之间会有<br/>，刚好适合用来分段落
    content = div_text.split('<br/>')[1:]
    #再将过多的\n去除
    content = "".join(content).split('\n')[:-2]
    
    #获得章节名、文章名、以及来源
    title = chapter_soup.title.text
    mainTitle = title.split('_')[1]
    source = title.split('_')[2]
    chapterTitle = title.split('_')[0]
    
    print('---Fetched the contents---')
    
    return content, chapterTitle


## 2.写入word文档和同名txt文件

In [10]:
def download(text_list, mainTitle):
    begin = time.time()
    if cache:
        content, chapterTitle = write_to_doc(cache)
        mainTitle = mainTitle + "更新自" + chapterTitle
    if cache:
        index = text_list.index(cache)
        if len(text_list) - 1 == index:
            print("没更新")
            return
        text_list = text_list[index:]
    #新建一个docx文件
    file=docx.Document()
    #新建一个txt文件
    file_handle = open('{}.txt'.format(mainTitle), mode = 'w')
    #写入主标题和作者名
    file.add_heading("{}".format(mainTitle),level=1)
    file.add_heading("作者:{}".format(author),level=2)
    file.add_page_break()
    count = 0
    total = len(text_list)
    
    for i in text_list:
        #count 用来计数，数字为章节数
        count  += 1
        print('-------------Doing chapter: {}/{}------------'.format(count, total))

        content, chapterTitle = write_to_doc(i)
        #写入段落标题
        file.add_heading("{}".format(chapterTitle),level=2)
        file_handle.write('\n')
        file_handle.write(chapterTitle)
        file_handle.write('\n')
        print('----begin to write chapter {} into doc----'.format(count))
        #将爬取回的内容一段一段写入docx
        for i in content:
            if i !=  ' ':
                file.add_paragraph(i)
                file_handle.write(i)
                file_handle.write('\n')
        print('---Saving for chapter {}!---'.format(count))
        #加入分页，每一章的开头都是全新的一页纸
        file.add_page_break()
        #保存docx文件
        file.save("{}.docx".format(mainTitle)) #输出的word名字
        #sleep(1)防止对服务器造成过大负担
        time.sleep(1)
        print('---Done for chapter: {}---\n'.format(count))
    file_handle.close()
    print('--------------------------------DONE!!!!!!!-------------------------')
    print('------------------uesd: {:.2f}s-------------------'.format(time.time() - begin))

In [11]:
download(text_list, mainTitle)

---Getting URL...---
-----URL 36232370.html used 1.40s-----
---Fetched the contents---
-------------Doing chapter: 1/19------------
---Getting URL...---
-----URL 36232370.html used 0.89s-----
---Fetched the contents---
----begin to write chapter 1 into doc----
---Saving for chapter 1!---
---Done for chapter: 1---

-------------Doing chapter: 2/19------------
---Getting URL...---
-----URL 36235578.html used 0.98s-----
---Fetched the contents---
----begin to write chapter 2 into doc----
---Saving for chapter 2!---
---Done for chapter: 2---

-------------Doing chapter: 3/19------------
---Getting URL...---
-----URL 36235579.html used 1.26s-----
---Fetched the contents---
----begin to write chapter 3 into doc----
---Saving for chapter 3!---
---Done for chapter: 3---

-------------Doing chapter: 4/19------------
---Getting URL...---
-----URL 36235580.html used 0.96s-----
---Fetched the contents---
----begin to write chapter 4 into doc----
---Saving for chapter 4!---
---Done for chapter: 4--

In [70]:
print(15127.34/60//60)
print(1512.34/60%60)

4.0
25.205666666666666


In [76]:
file_handle = open('/test/test.txt', mode = 'w')
file_handle.write('Hello World')
file_handle.write('\n')
file_handle.write('Hello World')
file_handle.close()

FileNotFoundError: [Errno 2] No such file or directory: '/test/test.txt'